In [ ]:
!pip install fuzzywuzzy

In [ ]:
from pyspark.sql.functions import col, levenshtein

# Renomear colunas do dataframe de filmes
df_movies_renamed = df_movies_IMDB.select(col("tituloPrincipal").alias("tituloPrincipal_movies"))

# Renomear colunas do dataframe de séries
df_series_renamed = df_series_IMDB.select(col("tituloPrincipal").alias("tituloPrincipal_series"))

# Realizar a comparação entre os títulos dos dataframes
df_similarity = df_movies_renamed.crossJoin(df_series_renamed) \
    .withColumn("title_similarity", levenshtein(col("tituloPrincipal_movies"), col("tituloPrincipal_series")))

# Filtrar os registros com títulos semelhantes e manter apenas os registros da tabela de séries
df_filtered = df_similarity.filter(col("title_similarity") <= 2)

# Inner join com a tabela de séries para manter apenas os registros com similaridade
df_result = df_series_IMDB.join(df_filtered, df_series_IMDB["tituloPrincipal"] == df_filtered["tituloPrincipal_series"], "inner")

# Exibir os resultados
df_result.show()

A função de levenshtein é uma função de similaridade de strings que mede a diferença entre duas strings com base no número mínimo de operações necessárias para transformar uma string na outra. Essas operações incluem inserção, exclusão e substituição de caracteres.

No caso do PySpark, a função levenshtein é implementada como uma função do módulo pyspark.sql.functions. Ela compara duas colunas de string e retorna a distância de Levenshtein entre elas.

A função de levenshtein é útil para comparar a similaridade entre strings e pode ser usada em várias aplicações, como a deduplicação de registros, correção ortográfica, pesquisa fuzzy, entre outros.

A função levenshtein considera todas as operações necessárias para transformar uma string na outra, incluindo inserções, exclusões e substituições de caracteres. Portanto, se houver caracteres especiais em suas strings, eles serão tratados corretamente e terão impacto no cálculo da distância de Levenshtein.

A distância de Levenshtein é calculada por meio de iterações recursivas, onde cada iteração representa uma operação que pode ser realizada para transformar uma string na outra. Existem três operações possíveis:

Inserção (Insert): Consiste em inserir um caractere em uma das strings para torná-las iguais. Isso aumenta a distância em 1.

Exclusão (Delete): Consiste em excluir um caractere de uma das strings para torná-las iguais. Isso também aumenta a distância em 1.

Substituição (Substitute): Consiste em substituir um caractere por outro em uma das strings para torná-las iguais. Isso aumenta a distância em 1.

Cada iteração da distância de Levenshtein é uma operação dessas três opções. O objetivo é encontrar a sequência de operações que minimize a distância total entre as strings.